## Darcy 3D

In [ ]:
from dolfinx import mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)

In [ ]:
from dolfin import *

class Bottom(SubDomain):
    def inside(self,x, on_boundary):
        return near(x[2],0) and on_boundary

class Top(SubDomain):
    def inside(self,x, on_boundary):
        return near(x[2],1) and on_boundary

def wall(x, on_boundary):
        return (near(x[0],0) or near(x[0],1) or near(x[1],0) or near(x[1],1)) and on_boundary

### MESH & FUNCTION SPACE ###

mesh    = UnitCubeMesh(10,10,10)

Qe      = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
Be      = FiniteElement("Bubble",   mesh.ufl_cell(), 4)
Ve      = VectorElement(NodalEnrichedElement(Qe, Be))
element = MixedElement(Ve,Qe)
W       = FunctionSpace(mesh, element)

n       = FacetNormal(mesh)
u,p     = TrialFunctions(W)
v,q     = TestFunctions(W)

In [ ]:
### SUB-DOMAINS ###

bottom = Bottom()
top    = Top()

sub_domains = MeshFunction('size_t', mesh, 1)
sub_domains.set_all(0)

bottom.mark(sub_domains, 1)
top.mark(sub_domains, 2)

ds = Measure('ds', domain=mesh, subdomain_data=sub_domains)

### BOUNDARY CONDITION ###

bc = DirichletBC(W.sub(0), Constant((0,0,0)), wall)

### VARIATIONAL FORMULATION ###

a =  (dot(u,v) - div(v)*p - div(u)*q)*dx
L = -Constant(1)*dot(n,v)*ds(1) + Constant(2)*dot(n,v)*ds(2)

w = Function(W)

solve(a == L, w, bc, solver_parameters={'linear_solver': 'mumps'})

file = File("flow.pvd")
file << w.split()[0]

file = File("pressure.pvd")
file << w.split()[1]

file = File("subdomains.pvd")
file << sub_domains

In [1]:
from mpi4py import MPI
from petsc4py import PETSc

import numpy as np

from basix.ufl import element, mixed_element
from dolfinx import fem, io, mesh
from dolfinx.fem.petsc import LinearProblem
from ufl import (Measure, SpatialCoordinate, TestFunctions, TrialFunctions,
                 div, exp, inner)

domain = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.quadrilateral)

k = 1
Q_el = element("BDMCF", domain.basix_cell(), k)
P_el = element("DG", domain.basix_cell(), k - 1)
V_el = mixed_element([Q_el, P_el])
V = fem.functionspace(domain, V_el)

(sigma, u) = TrialFunctions(V)
(tau, v) = TestFunctions(V)

x = SpatialCoordinate(domain)
f = 10.0 * exp(-((x[0] - 0.5) * (x[0] - 0.5) + (x[1] - 0.5) * (x[1] - 0.5)) / 0.02)

dx = Measure("dx", domain)
a = inner(sigma, tau) * dx + inner(u, div(tau)) * dx + inner(div(sigma), v) * dx
L = -inner(f, v) * dx


fdim = domain.topology.dim - 1
facets_top = mesh.locate_entities_boundary(domain, fdim, lambda x: np.isclose(x[1], 1.0))
Q, _ = V.sub(0).collapse()
dofs_top = fem.locate_dofs_topological((V.sub(0), Q), fdim, facets_top)


def f1(x):
    values = np.zeros((2, x.shape[1]))
    values[1, :] = np.sin(5 * x[0])
    return values


f_h1 = fem.Function(Q)
f_h1.interpolate(f1)
bc_top = fem.dirichletbc(f_h1, dofs_top, V.sub(0))


facets_bottom = mesh.locate_entities_boundary(domain, fdim, lambda x: np.isclose(x[1], 0.0))
dofs_bottom = fem.locate_dofs_topological((V.sub(0), Q), fdim, facets_bottom)


def f2(x):
    values = np.zeros((2, x.shape[1]))
    values[1, :] = -np.sin(5 * x[0])
    return values


f_h2 = fem.Function(Q)
f_h2.interpolate(f2)
bc_bottom = fem.dirichletbc(f_h2, dofs_bottom, V.sub(0))


bcs = [bc_top, bc_bottom]

problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu",
                                                      "pc_factor_mat_solver_type": "mumps"})
try:
    w_h = problem.solve()
except PETSc.Error as e:  # type: ignore
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

sigma_h, u_h = w_h.split()

with io.XDMFFile(domain.comm, "out_mixed_poisson/u.xdmf", "w") as file:
    file.write_mesh(domain)
    file.write_function(u_h)

In [2]:
w_h

Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, quadrilateral, 1, gll_warped, unset, False, float64, []), (2,)), 0), mixed element (Basix element (BDM, quadrilateral, 1, legendre, legendre, False, float64, []), Basix element (P, quadrilateral, 0, gll_warped, unset, True, float64, []))), 2)